## AI Game playing project

In [74]:
# prompt: generate chess game

import random

class ChessGame:
    def __init__(self, black_king_pos):
        self.board = [['.' for _ in range(8)] for _ in range(8)]
        self.board_black = [['.' for _ in range(8)] for _ in range(8)]
        self.current_player = 'white'
        self.black_king = self.position_to_index(black_king_pos,'add_black_king')
        self.pieces_black = []
        self.pieces_white = []
        self.possible_moves = dict()
        self.possible_moves = self.initiate_possible_moves()
        print(self.possible_moves)
        self.checked = False
        self.stalemated = False
        self.checkmated = False
        self.score = 0
        self.add_piece('k'+black_king_pos)
    
    def initiate_possible_moves(self):
        possible_moves = {'white':dict(),'black':dict()}
        for piece in ['P','R','N','B','Q','K']:
            possible_moves['white'][piece] = []
            possible_moves['black'][piece.lower()] = []
        # print(possible_moves)
        return possible_moves

    def print_board(self):
        print('  +---+---+---+---+---+---+---+---+')
        for row in range(8):
            print(8-row, end=' | ')
            for col in range(8):
                print(self.board[8-row-1][col], end=' | ')
            print('\n  +---+---+---+---+---+---+---+---+')
        print('    a   b   c   d   e   f   g   h')

    def print_black_board(self):
        print('  +---+---+---+---+---+---+---+---+')
        for row in range(8):
            print(8-row, end=' | ')
            for col in range(8):
                print(self.board_black[8-row-1][col], end=' | ')
            print('\n  +---+---+---+---+---+---+---+---+')
        print('    a   b   c   d   e   f   g   h')
    
    def position_to_index(self, inp, mode):
        if mode == 'add_black_king':
            col = ord(inp[0].lower())-97
            row = int(inp[1])-1
            return tuple([row,col])
        elif mode == 'add':
            piece = inp[0]
            col = ord(inp[1].lower())-97
            row = int(inp[2])-1
            return tuple([piece,row,col])
        else:
            # move
            if len(inp) == 3 or len(inp) == 4:
                # Pe7, Pe8Q
                piece = inp[0]
                col = ord(inp[1].lower())-97
                row = int(inp[2])-1
                moves = []
                promote = '_'
                if len(inp) == 4:
                    promote = inp[3]
                for move in self.possible_moves[self.current_player][piece]:
                    if row == move[2] and col == move[3] and promote == move[4]:
                        moves.append(move)
                if len(moves) ==  0:
                    print('Invalid move')
                elif len(moves) > 1:
                    print('Ambiguous move')
                    for m in moves:
                        print('move: ',tuple([piece]+list(m)))
                else:
                    print('move: ',tuple([piece]+list(moves[0])))
                    return tuple([piece]+list(moves[0]))
                return  None
            else:
                # len 5,6
                # Pe6e7, Pe7e8Q
                piece = inp[0]
                col = ord(inp[1].lower())-97
                row = int(inp[2])-1
                col2 = ord(inp[3].lower())-97
                row2 = int(inp[4])-1
                promote = '_'
                if len(inp) == 6:
                    promote = inp[5]
                print('move: ',tuple([piece,row,col,row2,col2,promote]))
                return tuple([piece,row,col,row2,col2,promote])
        
    def add_piece(self,order):
        piece,row,col = self.position_to_index(order,'add')
        if 0 <= row < 8 and 0 <= col <8 and piece in list(self.possible_moves['white'].keys())+list(self.possible_moves['black'].keys()):
            if piece == 'K' and 'K' in self.pieces_white:
                print('Only one white king can be added')
            elif piece == 'k' and 'k' in self.pieces_black:
                print('Only one black king can be added')
            elif self.board[row][col] == '.':
                self.board[row][col] = piece
                self.eval_board()
                if not self.checked:
                    self.print_black_board()
                    self.print_board()
                    if piece.islower():
                        self.pieces_black.append(piece)
                    else:
                        self.pieces_white.append(piece)
                    print('Successfully added '+ order)
                else:
                    self.board[row][col] = '.'
                    self.eval_board()
                    print('Black cannot be checked at start')
            else:
                print('Piece already exists at that position')
        else:
            print('Invalid position or piece')
    
    def eval_board(self):
        # {'white':{'P':[[from_row,from_col,to_row,to_col,promote],[]...]}}}
        possible_moves = self.initiate_possible_moves()
        new_black_board = [['.' for _ in range(8)] for _ in range(8)]
        new_white_board = [['.' for _ in range(8)] for _ in range(8)]
        # Mark black king's position
        for row in range(8):
            for col in range(8):
                if self.board[row][col] == 'k':
                    bkr = row
                    bkc = col
                    self.black_king = tuple([row,col])
        # Mark black king's surrounding
        moveset = [[_,__] for _ in [1,0,-1] for __ in [1,0,-1] if _ != 0 or __ != 0]
        for m in moveset:
            if 0 <= bkr + m[0] < 8 and 0 <= bkc + m[1] < 8:
                new_white_board[bkr+m[0]][bkc+m[1]] = 'o'
                # print('black mark', bkr+m[0],bkc+m[1])
        
        for row in range(8):
            for col in range(8):
                if self.board[row][col] == '.':
                    continue

                elif self.board[row][col] == 'P':
                    # Pawn
                    if 0 <= row + 1 < 7:
                        if self.board[row+1][col] == '.':
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col,'_']))
                        if self.board[row+1][col+1].islower() and 0 <= col + 1 < 8:
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col+1,'_']))
                        if self.board[row+1][col-1].islower() and 0 <= col - 1 < 8:
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col-1,'_']))
                    elif row + 1 == 7:
                        #promote
                        if self.board[row+1][col] == '.':
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col,'N']))
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col,'Q']))
                        if col + 1 < 8 and self.board[row+1][col+1].islower():
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col+1,'N']))
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col+1,'Q']))
                        if 0 <= col - 1 and self.board[row+1][col-1].islower():
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col-1,'N']))
                            possible_moves['white']['P'].append(tuple([row,col,row+1,col-1,'Q']))
                    if 0 <= row + 1 < 8:
                        if col + 1 < 8:
                            new_black_board[row+1][col+1] = 'x'
                        if col - 1 >= 0:
                            new_black_board[row+1][col-1] = 'x'

                elif self.board[row][col] == 'R':
                    # Rook
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['R'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col
                        i = 1
                        n_row = row
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['R'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row
                            n_col = col + i*dir

                elif self.board[row][col] == 'N':
                    moveset = [[_,__] for _ in [2,1,-1,-2] for __ in [2,1,-1,-2] if abs(_)!=abs(__)]
                    for m in moveset:
                        n_row = row + m[0]
                        n_col = col + m[1]
                        if 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['N'].append(tuple([row,col,n_row,n_col,'_']))

                elif self.board[row][col] == 'B':
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['B'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col - i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['B'](tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col - i*dir

                elif self.board[row][col] == 'Q':
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col
                        i = 1
                        n_row = row
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col - i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col - i*dir

                elif self.board[row][col] == 'K':
                    moveset = [[_,__] for _ in [1,0,-1] for __ in [1,0,-1] if _ != 0 or __ != 0]
                    for m in moveset:
                        n_row = row + m[0]
                        n_col = col + m[1]
                        if 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper() and new_white_board[n_row][n_col] != 'o':
                                possible_moves['white']['Q'].append(tuple([row,col,n_row,n_col,'_']))

        moveset = [[_,__] for _ in [1,0,-1] for __ in [1,0,-1] if _ != 0 or __ != 0]
        for m in moveset:
            n_row = bkr + m[0]
            n_col = bkc + m[1]
            if 0 <= n_row < 8 and 0 <= n_col < 8:
                if not self.board[n_row][n_col].isupper() and new_black_board[n_row][n_col] != 'x':
                    possible_moves['black']['k'].append(tuple([bkr,bkc,n_row,n_col,'_']))
        self.possible_moves = possible_moves
        self.board_black = new_black_board
        self.checked = self.board_black[self.black_king[0]][self.black_king[1]] == 'x'
        if self.checked:
            self.board_black[self.black_king[0]][self.black_king[1]] = '*'
        else:
            self.board_black[self.black_king[0]][self.black_king[1]] = 'k'
        if not self.checked and len(possible_moves['black']['k']) == 0 and self.current_player == 'white':
            self.score = -10000
        else:
            self.score = 8 - len(possible_moves['black']['k'])
        self.stalemated = not self.checked and len(possible_moves['black']) == 0
        self.checkmated = self.checked and len(possible_moves['white']) == 0
        print(self.possible_moves['white'])
        print(self.possible_moves['black'])
        

    def make_move(self, order):
        if order[0].islower() and self.current_player == 'white':
            print('White\'s turn')
            return
        elif order[0].isupper() and self.current_player == 'black':
            print('Black\'s turn')
            return
        t = self.position_to_index(order,'move')
        if t == None:
            print('Invalid move, please try again')
            return
        else:
            piece, from_row, from_col, to_row, to_col, promote = t
        if piece in self.possible_moves[self.current_player].keys():
            if tuple([from_row,from_col,to_row,to_col,promote]) in self.possible_moves[self.current_player][piece]:
                self.board[from_row][from_col] = '.'
                if promote == '_':
                    self.board[to_row][to_col] = piece
                else:
                    self.board[to_row][to_col] = promote
                if self.current_player == 'white':
                    self.current_player = 'black'
                else:
                    self.current_player = 'white'
        
        self.eval_board()
        self.print_black_board()
        self.print_board()
        print('score: ', self.score)
    def get_random_move(self):
        possible_moves = []
        for piece in self.possible_moves[self.current_player].keys():
            for move in self.possible_moves[self.current_player][piece]:
                possible_moves.extend(move)
        return random.choice(possible_moves)
        

def main():

    # initiate game with black king position
    game = ChessGame('a6')
    game.add_piece('Qe4')
    game.add_piece('Ph7')
    while not game.stalemated and not game.checkmated:
        game.make_move('Qe3')
        game.make_move('kb6')
        game.make_move('kb5')
        game.make_move('Ph8Q')
        break




    # game.print_board()
    # while True:
    #     move = game.get_random_move()
    #     if move is None:
    #         print("No more moves possible. Game Over.")
    #         break
    #     game.make_move(move)
    #     game.print_board()


if __name__ == "__main__":
    main()

{'white': {'P': [], 'R': [], 'N': [], 'B': [], 'Q': [], 'K': []}, 'black': {'p': [], 'r': [], 'n': [], 'b': [], 'q': [], 'k': []}}
{'P': [], 'R': [], 'N': [], 'B': [], 'Q': [], 'K': []}
{'p': [], 'r': [], 'n': [], 'b': [], 'q': [], 'k': [(5, 0, 6, 1, '_'), (5, 0, 6, 0, '_'), (5, 0, 5, 1, '_'), (5, 0, 4, 1, '_'), (5, 0, 4, 0, '_')]}
  +---+---+---+---+---+---+---+---+
8 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
7 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
6 | k | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
5 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
4 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
3 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
2 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
1 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h
  +---+---+-

In [71]:

moveset = [[_,__] for _ in [1,0,-1] for __ in [1,0,-1] if _ != 0 or __ != 0]
print(moveset)

[[1, 1], [1, 0], [1, -1], [0, 1], [0, -1], [-1, 1], [-1, 0], [-1, -1]]
